# Representação de dados

Para que uma informação seja representada em bytes, é preciso que uma convenção de representação seja seguida.

## Conversões

Nos exemplos seguintes, são apresentadas algumas alternativas para a codificação:

* Cadeias de caracteres: uso de conjunto de caracteres UTF-8
* Valores inteiros: uso de valor binário com sinal de 4 bytes de comprimento, *big endian*
* Valores reais: uso do padrão IEEE 754 de dupla precisão (8 bytes)

In [ ]:
# Cadeias de caracteres
cadeia = "Computação"
bytes_cadeia = bytes(cadeia, "utf-8")
print(f"{cadeia} --> {bytes_cadeia}")

# Inteiros
inteiro = 2500
bytes_inteiro = inteiro.to_bytes(4, "big", signed = True)
print(f"{inteiro} --> {bytes_inteiro}")

# Reais
from struct import pack
real = -3.1415666
bytes_real = pack("d", real)
print(f"{real} --> {bytes_real}")

As conversões inversas permitem recuperar os valores originais.

In [ ]:
# Cadeia
print(f"{bytes_cadeia} --> {bytes_cadeia.decode('utf-8')}")

# Inteiro
print(f"{bytes_inteiro} --> {int.from_bytes(bytes_inteiro, 'big', \
                                            signed = True)}")

# Real
from struct import unpack
print(f"{bytes_real} --> {unpack('d', bytes_real)[0]}")

## Dados em arquivos

Agora é hora de gravar os dados em um arquivo (binário, é claro)!

Para tanto, são gravadas as representações dos dados em bytes.

In [ ]:
# Criação de um arquivo binário
try:
  arquivo = open("dados.dat", "wb")
except IOError as excecao:
  print(f"Erro de acesso: {excecao}")
else:
  # Gravação dos dados binários
  comprimento = arquivo.write(bytes_cadeia)
  print(f"{comprimento} bytes escritos")
  comprimento = arquivo.write(bytes_inteiro)
  print(f"{comprimento} bytes escritos")
  comprimento = arquivo.write(bytes_real)
  print(f"{comprimento} bytes escritos")

  # Encerramento do acesso ao arquivo
  arquivo.close()
  print()

# Visualização do conteúdo do arquivo
!ls -l dados.dat
!hd dados.dat

Agora, a recuperação dos dados, incluindo as conversões para a representação original.

In [ ]:
# Acesso ao arquivo binário
try:
  arquivo = open("dados.dat", "rb")
except IOError as excecao:
  print(f"Erro de acesso: {excecao}")
else:
  bytes_dados = arquivo.read(12)  # 12 bytes da string
  print(bytes_dados.decode("utf-8"))

  bytes_dados = arquivo.read(4)  # 4 bytes do inteiro
  print(int.from_bytes(bytes_dados, "big", signed = True))

  bytes_dados = arquivo.read(8)  # 8 bytes do real
  print(unpack("d", bytes_dados)[0])

## Exemplo: arquivo de nomes

Segue um exemplo de criação de um arquivo de nomes.

In [ ]:
# Criação de um arquivo binário
try:
  arquivo = open("nomes.dat", "wb")
except IOError as excecao:
  print(f"Erro de acesso: {excecao}")
else:
  # Entrada de dados até linha vazia
  nome = input("Nome: ")
  while nome != "":
    # Conversão e gravação
    bytes_nome = bytes(nome, "utf-8")
    comprimento = arquivo.write(bytes_nome)
    print(f"{comprimento} bytes escritos")

    # Próxima leitura
    nome = input("Nome: ")

  # Encerramento do acesso ao arquivo
  arquivo.close()
  print()

In [ ]:
# Visualização do conteúdo do arquivo
!ls -l nomes.dat
!hd nomes.dat
!cat nomes.dat

# Campos

Campo é a menor unidade de informação.

Para programar com campos, foram implementadas classes para facilitar as conversões, gravações e leituras em arquivos.

A implementação do pacote [estrutarq](https://github.com/jandermoreira/estrutarq) está disponível no GitHub.

In [ ]:
!# Execute para baixar e atualizar o código disponível!
!git clone http://github.com/jandermoreira/estrutarq || \
  echo "Já foi executado antes? Basta baixar só uma vez..."
!cd estrutarq; git pull

## Campos com terminadores

Os limites de um campo podem ser determinados pelo uso de um terminador que indique o fim dos bytes de dados. A representação dos dados é sempre textual, inclusive para campos núméricos.

In [ ]:
# Exemplos de campos em formato textual com terminadores
from estrutarq.campo import CampoCadeiaTerminador, CampoRealTerminador

# Campo cadeia de caracteres
campo_nome = CampoCadeiaTerminador()  # terminador é 0x00
campo_nome.valor = "José João Victório Magalhães"
print(f"Nome: {campo_nome.valor}")
print(f" * {campo_nome.dado_formatado}")

# Campo real
campo_salario = CampoRealTerminador()
campo_salario.valor = 5889.20
print(f"Salário: {campo_salario.valor:.2f}")
print(f" * {campo_salario.dado_formatado}")

## Campos prefixados pelo comprimento

O prefixo adotado para o comprimento é de dois bytes (16 bits), usando um número inteiro binário sem sinal. Desta forma, o comprimento máximo é $2^{16}$ bytes.

In [ ]:
# Exemplos de campos em formato textual com prefixo de comprimento em binário
from estrutarq.campo import CampoIntPrefixado, CampoRealPrefixado, CampoCadeiaPrefixado

# Campo inteiro
campo_idade = CampoIntPrefixado()
campo_idade.valor = -987654779
print(f"Idade: {campo_idade.valor}")
print(f" * {campo_idade.dado_formatado}")

# Campo cadeia
campo_cadeia = CampoCadeiaPrefixado()
campo_cadeia.valor =  "A" * 2048  # cadeia longa (2KiB)
print(f"Idade: {campo_cadeia.valor}")
print(f" * {campo_cadeia.dado_formatado}")

# Campo real
campo_salario = CampoRealPrefixado()
campo_salario.valor = 5889.20
print(f"Salário: {campo_salario.valor:.2f}")
print(f" * {campo_salario.dado_formatado}")

## Campos de comprimento fixo

Um campo de comprimento fixo é definido por um comprimento prederminado de bytes, o que pode incorrer em truncamentos.

O campo fica com uma área útil (com os dados) e com possível fragmentação (disperdício). A parte não usada usa um byte de preenchimento para indicar que não é dado válido. O preenchimento dos campos é feito com o byte 0xFF.

In [ ]:
# Exemplos de campos em formato textual com comprimento fixo
from estrutarq.campo import CampoDataFixo, CampoIntFixo, CampoRealFixo

# Campo inteiro
campo_idade = CampoIntFixo(15)  # 15 bytes
campo_idade.valor = -987654
print(f"Idade: {campo_idade.valor}")
print(f" * {campo_idade.dado_formatado}")

# Campo data
campo_data = CampoDataFixo()  # datas usam sempre 10 bytes de comprimento
campo_data.valor = "2022-06-14"
print(f"Data: {campo_data.valor}")
print(f" * {campo_data.dado_formatado}")

# Campo real
campo_salario = CampoRealFixo(12)  # 12 bytes
campo_salario.valor = 5889.20
print(f"Salário: {campo_salario.valor:.2f}")
print(f" * {campo_salario.dado_formatado}")

## Campos binários

Para alguns campos, a representação binária é mais natural: é o caso de inteiros e reais. Tempo (hora e data) também podem usar a representação binária.

In [ ]:
# Exemplos de campos em formato binário
from estrutarq.campo import CampoHoraBinario, CampoIntBinario, CampoRealBinario


# Campo inteiro
campo_idade = CampoIntBinario()
campo_idade.valor = -987654
print(f"Idade: {campo_idade.valor}")
print(f" * {campo_idade.dado_formatado}")

# Campo data
campo_hora = CampoHoraBinario()
campo_hora.valor = "13:45:00"
print(f"Hora: {campo_hora.valor}")
print(f" * {campo_hora.dado_formatado}")

# Campo real
campo_salario = CampoRealBinario()
campo_salario.valor = 5889.20
print(f"Salário: {campo_salario.valor:.2f}")
print(f" * {campo_salario.dado_formatado}")

# Arquivos com campos

Uma vez escolhida a forma de representação dos dados, um arquivo com os campos pode ser criado, obedecidas as formas de organização de campos.

Independentemente da forma de organização ou se os dados são textuais ou binários, os aquivos são sempre acessados como binários, lendo ou escrevendo sequências de bytes.

Segue, assim, um exemplo de criação de um arquivo para guardar uma matriz de valores reais.

O arquivo é organizado da seguinte forma:
* Dois valores inteiros textuais com terminador, sendo o primeiro o número de linhas e o segundo, o número de colunas
* Uma descrição textual da matriz (prefixada pelo comprimento)
* Os valores reais de cada elemento da matriz, em formato binário


In [ ]:
# Criação de uma matriz
import numpy

numero_linhas = 10
numero_colunas = 6
matriz = numpy.random.rand(numero_linhas, numero_colunas) * 100
print(matriz)

In [ ]:
# Escrita da matriz em um arquivo
from estrutarq.campo import CampoIntTerminador, CampoCadeiaPrefixado
try:
  arquivo = open("matriz", "wb")
except IOError as excecao:
  print(f"Erro: {excecao}")
else:
  # Dimensões
  dimensao = CampoIntTerminador()
  dimensao.valor = numero_linhas
  dimensao.escreva(arquivo)
  dimensao.valor = numero_colunas
  dimensao.escreva(arquivo)
  print("Dimensões")

  # Comentário
  comentario = CampoCadeiaPrefixado()
  comentario.valor = "Criado por mim, hoje"
  comentario.escreva(arquivo)
  print("Comentário")

  # Elementos
  elemento = CampoRealBinario()
  for i in range(numero_linhas):
    for j in range(numero_colunas):
      elemento.valor = matriz[i][j]
      elemento.escreva(arquivo)
  print("Elementos")

  arquivo.close()

In [ ]:
!hd matriz

Finalmente, a recuperação da matriz a partir do arquivo

In [ ]:
# Leitura da matriz em um arquivo
try:
  arquivo = open("matriz", "rb")
except IOError as excecao:
  print(f"Erro: {excecao}")
else:
  # Dimensões
  dimensao = CampoIntTerminador()
  dimensao.leia(arquivo)
  numero_linhas = dimensao.valor
  dimensao.leia(arquivo)
  numero_colunas = dimensao.valor
  print(f"Matriz: {numero_linhas} x {numero_colunas}")

  # Comentário
  comentario = CampoCadeiaPrefixado()
  comentario.leia(arquivo)
  print(f"Comentário: {comentario.valor}")

  # Criação da matriz e leitura dos elementos
  nova_matriz = numpy.zeros((numero_linhas, numero_colunas))
  elemento = CampoRealBinario()
  for i in range(numero_linhas):
    for j in range(numero_colunas):
      elemento.leia(arquivo)
      nova_matriz[i][j] = elemento.valor
    print()

  arquivo.close()

  print(nova_matriz)